### Create awards from backfill table `openalex.mid.work_funder`

In [0]:
%sql
CREATE OR REPLACE TABLE openalex.awards.backfill_awards
USING delta
AS
WITH awards AS (
  SELECT DISTINCT funder_id, lower(award_id) as normalized_award_id, award_id as funder_award_id
  FROM openalex.mid.work_funder
  LATERAL VIEW EXPLODE(award_ids) AS award_id
),
funders AS (
  SELECT 
    funder_id,
    struct(
      concat('https://openalex.org/F', funder_id) as id,
      display_name,
      ror_id
    ) as funder
  FROM openalex.common.funder
)
SELECT 
  concat(f.funder_id, ':', a.normalized_award_id) as id,
  f.funder_id,
  f.funder,
  a.funder_award_id,
  current_timestamp() as updated_date,
  current_timestamp() as created_date
FROM awards a
JOIN funders f ON f.funder_id = a.funder_id